## Setup a classification experiment

In [1]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]
# df = df.sample(frac=0.01, random_state=1)
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #Turning response into 0 and 1

# We have to transform categorical variables to use sklearn models
X_enc = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_enc.columns)

seed = 1  
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=seed)

## Train a blackbox classification system

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

#Blackbox system can include preprocessing, not just a classifier!
pca = PCA()
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

blackbox_model = Pipeline([('pca', pca), ('rf', rf)])
blackbox_model.fit(X_train, y_train)

Pipeline(steps=[('pca', PCA()), ('rf', RandomForestClassifier(n_jobs=-1))])

## Show blackbox model performance

In [3]:
from interpret import show
from interpret.perf import ROC

blackbox_perf = ROC(blackbox_model.predict_proba).explain_perf(X_test, y_test, name='Blackbox')
show(blackbox_perf)

C:\Users\kevyh\CS_Homework\cs279\interactiveml\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\kevyh\CS_Homework\cs279\interactiveml\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


<!-- http://127.0.0.1:7001/3029488851920/ -->

## Local Explanations: How an individual prediction was made

In [4]:
from interpret.blackbox import LimeTabular
from interpret import show

#Blackbox explainers need a predict function, and optionally a dataset
lime = LimeTabular(predict_fn=blackbox_model.predict_proba, data=X_train, random_state=1)

#Pick the instances to explain, optionally pass in labels if you have them
lime_local = lime.explain_local(X_test[:5], y_test[:5], name='LIME')

show(lime_local)

C:\Users\kevyh\CS_Homework\cs279\interactiveml\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\kevyh\CS_Homework\cs279\interactiveml\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\kevyh\CS_Homework\cs279\interactiveml\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\kevyh\CS_Homework\cs279\interactiveml\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\kevyh\CS_Homework\cs279\interactiveml\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\kevyh\CS_Ho

<!-- http://127.0.0.1:7001/3029558225568/ -->

In [5]:
from interpret.blackbox import ShapKernel
import numpy as np

background_val = np.median(X_train, axis=0).reshape(1, -1)
shap = ShapKernel(predict_fn=blackbox_model.predict_proba, data=background_val, feature_names=feature_names)
shap_local = shap.explain_local(X_test[:5], y_test[:5], name='SHAP')
show(shap_local)

X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names


  0%|          | 0/5 [00:00<?, ?it/s]

X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names


<!-- http://127.0.0.1:7001/3029725927824/ -->

## Global Explanations: How the model behaves overall

In [6]:
from interpret.blackbox import MorrisSensitivity

sensitivity = MorrisSensitivity(predict_fn=blackbox_model.predict_proba, data=X_train)
sensitivity_global = sensitivity.explain_global(name="Global Sensitivity")

show(sensitivity_global)

X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names


<!-- http://127.0.0.1:7001/3029556452976/ -->

In [7]:
from interpret.blackbox import PartialDependence

pdp = PartialDependence(predict_fn=blackbox_model.predict_proba, data=X_train)
pdp_global = pdp.explain_global(name='Partial Dependence')

show(pdp_global)

X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid fea

X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid fea

X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid fea

<!-- http://127.0.0.1:7001/3029607255248/ -->

## Compare them all in the Dashboard

In [8]:
show([blackbox_perf, lime_local, shap_local, sensitivity_global, pdp_global])

<!-- http://127.0.0.1:7001/3029608506560/ -->
 Open in new window